In [11]:
from semantic_router import Route, RouteLayer
from semantic_router.encoders import HuggingFaceEncoder
from semantic_router.llms import LlamaCppLLM
from llama_cpp import Llama
from semantic_router.utils.function_call import get_schema

import datetime


In [32]:
import json
import os
import redis
from datetime import datetime

redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379')
redis_db = redis.from_url(redis_url)

class Utils:

    @staticmethod
    def get_period() -> dict:
        return {'date': '2024-01-01'}
        #return ', '.join(user_obj['dates']) if user_obj['dates'] else redis_db.get(user_obj['userId'])

    @staticmethod
    def put_period(iso_time: str) -> datetime:
        """
            Inserts a new period entry into the database. 
            param iso_time: The ISO time to use in the period entry
            type timezone: str
            return: Python datetime object representing period time
        """
        periods = []
        period_date = datetime.datetime.fromisoformat (iso_time)
        periods.append (period_date)
        # redis_db.set(user_obj['userId'], periods)
        return period_date


In [30]:
put_period_schema = get_schema(Utils.put_period)
get_period_schema = get_schema(Utils.get_period)    

put_route = Route(
    name="put_period",
    utterances=[
        "I just had my period on 12th of May 2024",
        "Enter a new period entry for 25th of June 2023",
        "Add a new period entry for today the 1st of January 2023"],
    function_schema=put_period_schema)

get_route = Route(
    name="get_period", utterances=[
        "When was my last period?",
        "Tell me about my last period",
        "when was my last period",
        "get the details of my last period entry"])

routes = [get_route, put_route]

_llm = Llama(model_path="mistral-7b-instruct-v0.2.Q4_0.gguf", n_gpu_layers=-1, n_ctx=8000)
encoder = HuggingFaceEncoder (name="intfloat/e5-large-v2", device="mps" )
llm = LlamaCppLLM(name="Mistral-7B-v0.2-Instruct", llm=_llm, max_tokens=None)
rl = RouteLayer(encoder=encoder, routes=routes, llm=_llm)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 lla

In [33]:
def process_user_input( input_: str):
    out = rl (input_)
    print(out)
    # if out.name =="put_period":
    #     return Utils.put_period(**out.function_call)
    # elif out. name == "get_period":
    
    #     return Utils.get_period()
    # else:
    #     return "No matching routes"

process_user_input( "Add a new period for today 12th of May")
process_user_input( "When was my last period?")


name='put_period' function_call=None similarity_score=None
name='get_period' function_call=None similarity_score=None
